### ***Multilayer Perceptron and Backpropagation***

In [291]:
''' Import all important library '''
import torch
import torch.nn as nn
import torch.optim as gredient
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [292]:
Device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device Name is {Device}')

Device Name is cpu


### ***Data Pipeline***

In [293]:
Transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.1307),
        std=(0.3181)
    )
])
'''
Compose = all number convert between (0 - 1)[pixel_num / 255]
standard division
'''

'\nCompose = all number convert between (0 - 1)[pixel_num / 255]\nstandard division\n'

In [294]:
Train_Dataset = datasets.MNIST(
    root='/Users/mahadiur/Desktop/Deep Learning Forward and Backpropagation/Data',
    train=True,
    download=True,
    transform=Transformation
)

Test_Dataset = datasets.MNIST(
    root='/Users/mahadiur/Desktop/Deep Learning Forward and Backpropagation/Data',
    train=False,
    download=True,
    transform=Transformation
)

In [295]:
print(type(Train_Dataset))

<class 'torchvision.datasets.mnist.MNIST'>


### ***Dataloader***

In [296]:
Train_Dataloader = DataLoader(
    Train_Dataset,
    batch_size=32,
    shuffle=True
)

Test_Dataloader = DataLoader(
    Test_Dataset,
    batch_size=32,
    shuffle=False
)

### ***Perceptron***

In [297]:
class Perceptron(nn.Module):
    def __init__(self, input_size):
        super(Perceptron, self).__init__()
        self.weights = nn.Parameter(torch.randn(input_size))
        self.bias = nn.Parameter(torch.randn(1))
    
    def forward(self, x):
        sumation = x @ self.weights + self.bias
        return sumation

In [298]:
''' Testing '''
input_size = 28 * 28
shuffle = torch.randn(1, input_size)

perceptron = Perceptron(
    input_size=input_size
)

output = perceptron(shuffle)
print(output)

tensor([20.3762], grad_fn=<AddBackward0>)


In [299]:
'''
Some import term for image classification:
- Batch Size (8, 32, 64, 128)
- epoch
- Step 
MNIST Dataset :
- Grayscale image 
- pixel(28 * 28)
- 60000 images in MNIST dataset
'''

'\nSome import term for image classification:\n- Batch Size (8, 32, 64, 128)\n- epoch\n- Step \nMNIST Dataset :\n- Grayscale image \n- pixel(28 * 28)\n- 60000 images in MNIST dataset\n'

### ***Activation Function***

In [300]:
'''
Activation Function: RELU (Rectified Linear Unit)
Function: max(0, z)  
'''
class RELU(nn.Module):
    def __init__(self):
        super(RELU, self).__init__()
    def forward(self, x):
        return torch.maximum(torch.tensor(0.0), x)

In [301]:
relu = RELU()
output_relu = relu(output)
print(output_relu)

tensor([20.3762], grad_fn=<MaximumBackward0>)


### ***Perceptron Hidden Layer***

In [302]:
''' Perceptron Hidden Layer '''
class Linear(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.perceptron = nn.ModuleList([
            Perceptron(input_size) for _ in range(output_size)
        ])
    def forward(self, x):
        outputs = [
            perceptron(x) for perceptron in self.perceptron
        ]
        final_output =torch.stack(outputs, dim=1)
        return final_output


# Testing
linear = Linear(input_size,10) 
output = linear(shuffle)
print(output)
print(output.shape)

tensor([[ 29.1928,  -7.9388,  31.4675, -38.9871,  12.4125,  -3.6326, -15.6738,
         -43.8678, -34.1031, -18.2419]], grad_fn=<StackBackward0>)
torch.Size([1, 10])


### ***Digit Classifiar betweet (1, 10)***

In [303]:
class Digitclassifiar(nn.Module):
    def __init__(self, input_size=28 * 28, output_size=10):
        super().__init__()
        self.layer1 = Linear(input_size, 512 )
        self.layer2 = Linear(512, 256)
        self.layer3 = Linear(256, 128)
        self.layer4 = Linear(128, output_size)
        self.relu = RELU()
    
    def forward(self, x):
        x = x.view(-1, input_size) # [1 ... 784]
        x = self.layer1(x) # [1 ... 256]
        x = self.relu(x) # activation function
        x = self.layer2(x) # [1...128]
        x = self.relu(x)
        x = self.layer3(x) # [1...64]
        x = self.relu(x)
        x = self.layer4(x) # [1...10]
        return x

In [304]:
Model = Digitclassifiar(input_size=28 * 28, output_size=10).to(Device)
output = Model(shuffle)
print(output.shape)
print(output)

torch.Size([1, 10])
tensor([[ 12779.0156, -11349.4111,   5532.1357,  30899.3223, -28465.6348,
         -39870.1523,  35060.9062, -43509.5703,  18479.3672,  -4481.2886]],
       grad_fn=<StackBackward0>)


### ***Run epochs***

In [305]:
optimizer = gredient.Adam(Model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()
epochs = 5

for epoch in range(epochs):
    Model.train()
    current_loss = 0.0
    for batch_num, (data, target) in enumerate(Train_Dataloader):
        data, target = data.to(Device), target.to(Device)
        outputs = Model(data)
        loss = loss_function(outputs, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        current_loss += loss.item()
        if epoch % 100 == 0:
            print(f'Epoch {epoch + 1} / {epochs} Step [{batch_num + 1 }/ {len(Train_Dataloader)}] and Loss {loss.item():3f}')


Epoch 1 / 5 Step [1/ 1875] and Loss 65851.984375
Epoch 1 / 5 Step [2/ 1875] and Loss 44668.511719
Epoch 1 / 5 Step [3/ 1875] and Loss 53792.867188
Epoch 1 / 5 Step [4/ 1875] and Loss 41878.628906
Epoch 1 / 5 Step [5/ 1875] and Loss 37646.906250
Epoch 1 / 5 Step [6/ 1875] and Loss 46546.054688
Epoch 1 / 5 Step [7/ 1875] and Loss 42515.574219
Epoch 1 / 5 Step [8/ 1875] and Loss 36944.386719
Epoch 1 / 5 Step [9/ 1875] and Loss 41717.558594
Epoch 1 / 5 Step [10/ 1875] and Loss 23514.218750
Epoch 1 / 5 Step [11/ 1875] and Loss 35586.023438
Epoch 1 / 5 Step [12/ 1875] and Loss 41187.980469
Epoch 1 / 5 Step [13/ 1875] and Loss 21226.761719
Epoch 1 / 5 Step [14/ 1875] and Loss 25199.167969
Epoch 1 / 5 Step [15/ 1875] and Loss 27911.410156
Epoch 1 / 5 Step [16/ 1875] and Loss 29509.248047
Epoch 1 / 5 Step [17/ 1875] and Loss 20679.224609
Epoch 1 / 5 Step [18/ 1875] and Loss 21668.265625
Epoch 1 / 5 Step [19/ 1875] and Loss 26430.544922
Epoch 1 / 5 Step [20/ 1875] and Loss 24962.333984
Epoch 1 /

In [306]:
print(loss)

tensor(195.3880, grad_fn=<NllLossBackward0>)


### ***Model Accuracy***

In [307]:
Model.eval()
total = 0
Actual_val = 0

for data, target in Test_Dataloader:
    data, target = data.to(Device), target.to(Device)
    outputs = Model(data)    
    _, predict_val = torch.max(outputs.data, 1)
    total += target.size(0)
    Actual_val += (predict_val == target).sum().item()

Accuracy = 100 * Actual_val / total
print(f'Model Accuracy is {Accuracy}')


Model Accuracy is 94.73


# *Thank You*